In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
absorption_rates = {
    'Atlanta': (-0.022588977749568005, -0.00732961656528534),
    'Austin': (-0.053831439593598505, -0.026366892164422204),
    'Baltimore': (-0.014402099922755261, -0.005147754659864611), 
    'Boston': (-0.029969134164771902, -0.011081917032427216),
    'Charlotte': (-0.029097258685895997, -0.034973901999187214),
    'Chicago': (-0.028978654311836275, -0.03531692861556808),  
    'Chicago Suburbs': (-0.009359460176861338, 0.0009917111992323018), 
    'Dallas/Ft Worth': (-0.009950265078867248, -0.0026829764313393584),
    'Denver':  (-0.030744320256725036, -0.019115843962804146),  
    'Detroit': (-0.014402099922755261, -0.005147754659864611), 
    'Houston': (-0.0033821995029679262, -0.0009973758041107166),
    'Los Angeles': (-0.024518875716068075, -0.01301887242119987),
    'Manhattan': (-0.02010481058626688, -0.03209074988074955),
    'Nashville': (-0.027813898586050868, -0.02360324645464407),
    'Northern New Jersey': (-0.004553724270000923, 0.010556213549418455),
    'Northern Virginia': (-0.009088414839254471, 0.0010518660979513053),
    'Orange County': (-0.009838022065392011, -0.007423543101770445),
    'Philadelphia': (-0.01824711379288069, -0.004704105034526766),
    'Phoenix': (-0.0246509729805061, 0.0005434075531365124),
    'Raleigh/Durham': (-0.03249189391395942, -0.0035755733395049882),
    'Salt Lake City': (-0.042753829685275434, -0.02923822771966973),
    'San Diego': (-0.018787468581848914, -0.0006801506864007474),
    'San Francisco': (-0.07203686728867384, -0.07698653830148533),
    'Seattle': (-0.04447649820603075, -0.025559230697896996),
    'South Bay/San Jose': (-0.037018952994871526, -0.026381363658888376),
    'South Florida': (-0.007124659865635555, 0.002390870201891718),
    'Maryland': (-0.008100961117428517, 0.0011767018414884812),
    'Tampa': (-0.028027310449950927, -0.0028264714609103556),
    'Washington DC': (-0.021611869757852228, -0.017655357434349584),
}

len(absorption_rates)

In [ ]:
# data
leases = pd.read_csv('Leases.csv')
prices = pd.read_csv('Price and Availability Data.csv')

In [ ]:
# edit columns
market_mapping = {
    'Downtown Chicago': 'Chicago',
    'Denver-Boulder': 'Denver',
    'Suburban Maryland': 'Maryland',
    'Raleigh-Durham': 'Raleigh/Durham',
    'Dallas-Ft. Worth': 'Dallas/Ft Worth',
    'Orange County (CA)': 'Orange County',
    'South Bay': 'South Bay/San Jose',
    'Washington DC': 'Washington D.C.'
}
prices['market'] = prices['market'].replace(market_mapping)

leases['date'] = leases['year'].astype(str) + '_' + leases['quarter']
prices['date'] = prices['year'].astype(str) + '_' + prices['quarter']

In [ ]:
# input
REGION = 'Midwest/Central'
MARKET = 'Washington DC'

In [ ]:
print(f"Collect data for the {MARKET} market in the {REGION} region")
region_data, market_data, market_data_from_price_dataset = break_up_data(REGION, MARKET)
dataset = market_data_from_price_dataset

In [ ]:
dataset, removed = drop_na_rows('direct_available_space', dataset)
print(f"Dropped {removed} empty rows")

In [ ]:
print(f'Split by quality')
A_dataset, O_dataset = split_by_quality(dataset)

In [ ]:
print('Add absorption rate column')
A_dataset = find_absorption_rates(A_dataset)
O_dataset = find_absorption_rates(O_dataset)

In [ ]:
print('Mean absorption rates')
A_mean_absorption = find_mean_absorption(A_dataset)
O_mean_absorption = find_mean_absorption(O_dataset)

In [ ]:
A_mean_absorption, O_mean_absorption

In [ ]:
print('Plot absorption rates')
plot_absorption_rates(A_dataset, 'A', MARKET)
plot_absorption_rates(O_dataset, 'O', MARKET)

In [ ]:
print('Plot availability rates')
plot_availability_proportion(A_dataset, 'A', MARKET)
plot_availability_proportion(O_dataset, 'O', MARKET)

In [ ]:
def break_up_data(REGION, MARKET):
    region   = leases[leases['region'] == REGION]
    market   = region[region['market'] == MARKET]
    market_p = prices[prices['market'] == MARKET]
    
    return (region, market, market_p)

In [ ]:
def drop_na_rows(column_name, dataset):
    original_count = len(dataset)
    dataset        = dataset.dropna(subset=[column_name])
    new_count =  len(dataset)

    removed_count = original_count - new_count
    return (dataset, removed_count)

In [ ]:
def split_by_quality(dataset):
    dataset_A = dataset[dataset['internal_class'] == 'A']
    dataset_O = dataset[dataset['internal_class'] == 'O']
    
    return (dataset_A, dataset_O)

In [ ]:
def find_absorption_rates(dataset):
    
    dataset['absorption_rate'] = 0.0

    for i in range(1, len(dataset)):
        prev_avail = dataset['availability_proportion'].iloc[i-1]
        curr_avail = dataset['availability_proportion'].iloc[i]

        # Positive values mean availability decreased (space was absorbed)
        # Negative values mean availability increased
        if prev_avail != 0:  # Avoid division by zero
            dataset.loc[dataset.index[i], 'absorption_rate'] = (prev_avail - curr_avail) / prev_avail
            
    return dataset

In [ ]:
def find_mean_absorption(dataset):
    return dataset['absorption_rate'].mean()

In [ ]:
def plot_absorption_rates(dataset, quality, market):
    plt.figure(figsize=(10, 6))
    plt.plot(dataset['date'], dataset['absorption_rate'], marker='o', color='green')
    plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)  # Add a line at y=0 for reference
    plt.xlabel('Quarter')
    plt.ylabel('Absorption Rate')
    plt.title(f'Quarterly Absorption Rate for {quality}-leases in {market}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_availability_proportion(dataset, quality, market):
    plt.figure(figsize=(10, 6))
    plt.plot(dataset['date'], dataset['availability_proportion'], marker='o')
    plt.xlabel('Quarter')
    plt.ylabel('Availability Proportion')
    plt.title(f'Availability Proportion over Time for {quality}-leases in {market}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()